In [ ]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 11.6 MB/s eta 0:00:00


In [ ]:
# 정상 동작 여부 확인
import konlpy
from konlpy.tag import Okt
# konlpy 테스트
okt = Okt()
text = '자연어 처리 입니다'
okt.nouns(text)   # 명사 추출

['자연어', '처리']

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/bigdataleeky/data/main/ratings_train.txt",
                 delimiter='\t', keep_default_na=False)
# delimiter : 구분자 , 공백이나 "NA","NULL" --> 결측치로 자동 변환
# label이 0과 1일 경우 이진분류, 0~5일 경우 다중분류
df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [ ]:
# 결측치 확인
# document에 결측치가 있음
# 결측치가 있는 행만 출력
df[df.isna().any(axis=1)]     # any는 어느 하나라도 true이면 true , all : 전부 true -> true

,id,document,label


In [ ]:
# 클래스 불균형을 확인
# label에 대한 내용을 알아보려면
df.label.value_counts()
# 0과 1, 긍정과 부정으로 나뉨

label
0    75173
1    74827
Name: count, dtype: int64

In [ ]:
# 데이터 분류
# 지도 학습
X_train = df.document.to_numpy()
y_train = df.label.values

In [ ]:
X_train[:3], y_train[:3]

(array(['아 더빙.. 진짜 짜증나네요 목소리', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나',
        '너무재밓었다그래서보는것을추천한다'], dtype=object),
 array([0, 1, 0]))

In [ ]:
# 데이터 크기 확인
len(X_train), len(y_train)

(150000, 150000)

In [ ]:
# 테스트 데이터 https://raw.githubusercontent.com/bigdataleeky/data/main/ratings_test.txt
# X_test, y_test

In [ ]:
df_test = pd.read_csv('https://raw.githubusercontent.com/bigdataleeky/data/main/ratings_test.txt',
                      delimiter='\t', keep_default_na=False)

X_test = df_test.document.to_numpy()
y_test = df_test.label.to_numpy()

In [ ]:
len(X_test), len(y_test)

(50000, 50000)

In [ ]:
# from konlpy import Okt      # TfidVectorizer 파서
from konlpy.tag import Okt

In [ ]:
# 형태소 분석      morphs(text)
  # 가장 작은 말의 단위       사랑해 = '사랑'+'해'
  # 자음 모음 어미 어간

# 품사 태깅       pos(text)
  # 동사 명사 형용사 접속사 부사 대명사 전치사...

# 명사 추출        nouns(text)

In [ ]:
# 명사 추춣 함수
okt_tag = Okt()

def okt_tokenizer(text):
  return okt_tag.nouns(text)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TfidVectorizer
  # Term Frequency-inverse Document Frequency
  # 텍스트를 벡터화 한다.
  # TF단어의 출현빈도 : 많을 수록 수치가 높다.
  # IDF 단어가 문장에서의 중요 : 문장에서 드물게 나타날수록 값이 커진다
  # TF-IDF는 두개를 곱함 : 특정 단어가 특정 문장에서 얼마나 중요한지

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

In [ ]:
okt_pipe = make_pipeline(TfidfVectorizer(tokenizer=okt_tokenizer),LogisticRegression())

okt_param_grid = {
    # 소문자: 객체 이름
    'tfidfvectorizer__min_df':[3,5,7], # 단어의 최소 등장 빈도수.. 이 값보다 작은 단어는 무시
    'tfidfvectorizer__ngram_range':[(1,1),(1,2),(1,3)],   # n-그램의 범위를 설정    n-그램: 연속된 단어의 조합 (1,1)(1,2)
                                                          # 넓게 설정하면 문맥을 고려한 특성을 더 잘 추출할 수 있음
    'logisticregression__C':[0.1, 1, 10]  # 각각의 값에 대해 모델을 테스트
}

# 최적의 매개변수 찾기
okt_grid = GridSearchCV(okt_pipe, param_grid=okt_param_grid, cv=3)    # 교차 검증 / 최상의 파라미터(하이퍼 파라미터 튜닝)
# cv는 각 하이퍼파라미터 조합에 대한 교차 검증의 폴드 수를 결정

# 데이터 100개만 선택해서 학습
okt_grid.fit(X_train[:1000], y_train[:1000])
print(f'best score:{okt_grid.best_score_}')
print(f'best parameter:{okt_grid.best_params_}')

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extr

best score:0.6440032847218476
best parameter:{'logisticregression__C': 0.1, 'tfidfvectorizer__min_df': 3, 'tfidfvectorizer__ngram_range': (1, 3)}


In [ ]:
X_test_okt = okt_grid.best_estimator_.named_steps['tfidfvectorizer'].transform(X_test)

In [ ]:
print(X_test_okt[:10])

  (2, 440)	0.5117581756875116
  (2, 439)	0.3927801514048838
  (2, 361)	0.35596181685631706
  (2, 317)	0.34475140464424947
  (2, 151)	0.426059935308107
  (2, 97)	0.3959013863839895
  (3, 337)	0.4390505808340317
  (3, 298)	0.4067178520125543
  (3, 174)	0.47328049882794926
  (3, 126)	0.49211858208003345
  (3, 104)	0.41909431745646275
  (4, 370)	0.4682408097085795
  (4, 300)	0.31675074702738515
  (4, 167)	0.43913288135695394
  (4, 127)	0.4184804775236264
  (4, 118)	0.43913288135695394
  (4, 11)	0.34585867537901066
  (5, 413)	0.3408814523880661
  (5, 313)	0.9208321515745974
  (5, 278)	0.18938844749429784
  (6, 239)	1.0
  (7, 252)	1.0
  (8, 319)	0.417942402826946
  (8, 278)	0.1866622093754098
  (8, 128)	0.36585771365328335
  (8, 121)	0.5187134537456999
  (8, 68)	0.36585771365328335
  (8, 5)	0.5037002949515925
  (9, 192)	0.42273000266213695
  (9, 143)	0.6199435743665652
  (9, 106)	0.6610365416910602


In [ ]:
X_test_okt = okt_grid.best_estimator_.named_steps['tfidfvectorizer'].transform(X_test)

In [ ]:
okt_grid.best_estimator_.named_steps['logisticregression'].score(X_test_okt, y_test)
#  score안에 순서 지켜야함

0.64308

In [ ]:
okt_grid.score(X_test, y_test)

0.64308

Mecab:c++로 작성된 경량의 형태소 분석기

In [ ]:
# from konlpy.tag import Mecab
# mecab = Mecab()
# text = '즐거운 수요일 입니다.'
# mecab.morphs()